In [ ]:
import requests
from datetime import datetime
from uuid import uuid4
import os
import sys
import pandas as pd


def parameters(date=None):
    if date is not None:
        params = {"format": "json", "query.term": f"AREA[LastUpdatePostDate]RANGE[{date},MAX]"}
    else:
        params = {"format": "json"}
    return params

def check_status_code(response):
    if response.status_code == 200:
        pass
    else:
        raise Exception("Bad Request")
    
def make_output_folder():
    output_folder = 'output'
    os.makedirs(output_folder, exist_ok=True)

def check_wrong_date(response):
    if response.empty:
        raise ValueError("Please rerun the entire pipeline with date before the current entered date because there are no entries after this date")

def add_uuid_time_cols(data, pipeline_start_timestamp, uuid):
    data["pipeline_start_timestamp"] = pipeline_start_timestamp
    data["pipeline_run_id"] = uuid
    return data

def create_parquet(response, i, pipeline_start_timestamp):
    response.to_parquet("output/parquet_{}_time_{}.gzip".format(str(i), pipeline_start_timestamp))

def download(params, date):
    pipeline_start_timestamp = datetime.now()
    uuid = str(uuid4())
    
    make_output_folder()
    base_url = "https://clinicaltrials.gov/api/v2/studies"
    response = requests.get(base_url, params=params)
    response_ = pd.DataFrame(response.json())
    check_status_code(response)
    check_wrong_date(response_)
    response_ = add_uuid_time_cols(response_, pipeline_start_timestamp, uuid)
    i = 1
    create_parquet(response_, i, pipeline_start_timestamp)
    next_page_token = str(response_['nextPageToken'][0])
    while next_page_token:
        i += 1
        try:
            if date:
                response_cont = requests.get(base_url, params={"format": "json", "pageToken": next_page_token,
                                                               "query.term": f"AREA[LastUpdatePostDate]RANGE[{date},MAX]"})
            else:
                response_cont = requests.get(base_url, params={"format": "json", "pageToken": next_page_token})
            check_status_code(response)
            response_cont = pd.DataFrame(response_cont.json())
            response_cont = add_uuid_time_cols(response_cont, pipeline_start_timestamp, uuid)
            create_parquet(response_cont, i, pipeline_start_timestamp)
            next_page_token = str(response_cont['nextPageToken'][0])
        except KeyError:
            next_page_token = None


date = input("Enter the date in YYYY-MM-DD format (or leave blank for all records): ")
parameter = parameters(date)
result = download(parameter, date)

In [1]:
script = """
/*
Input table is named as "SalesRevenue"
*/

-- This CTE assigns quarters to each sale based on their SalesID, Year, and ProductCategory.
-- For the year 2024, only Q1 and Q2 are assigned based on the sequence of the SalesID for each ProductCategory.
WITH SalesRevenueQuarters AS (
    SELECT 
        SalesID,
        Year,
        SalesAmount,
        ProductCategory,
        CASE 
            WHEN Year = 2024 THEN
                CASE 
                    WHEN ROW_NUMBER() OVER (PARTITION BY Year, ProductCategory ORDER BY SalesID) = 1 THEN 'Q1'
                    WHEN ROW_NUMBER() OVER (PARTITION BY Year, ProductCategory ORDER BY SalesID) = 2 THEN 'Q2'
                    ELSE NULL
                END
            ELSE 
                CASE 
                    WHEN ROW_NUMBER() OVER (PARTITION BY Year, ProductCategory ORDER BY SalesID) % 4 = 1 THEN 'Q1'
                    WHEN ROW_NUMBER() OVER (PARTITION BY Year, ProductCategory ORDER BY SalesID) % 4 = 2 THEN 'Q2'
                    WHEN ROW_NUMBER() OVER (PARTITION BY Year, ProductCategory ORDER BY SalesID) % 4 = 3 THEN 'Q3'
                    WHEN ROW_NUMBER() OVER (PARTITION BY Year, ProductCategory ORDER BY SalesID) % 4 = 0 THEN 'Q4'
                END
        END AS Quarter
    FROM SalesRevenue
),
-- This CTE calculates the total sales for each combination of Year, ProductCategory, and Quarter.
-- It sets the basis for pivoting the columns in next CTE.
SalesRevenuePivot AS (
    SELECT 
        Year,
        ProductCategory,
        Quarter,
        SUM(SalesAmount) AS TotalSales
    FROM SalesRevenueQuarters
    WHERE Quarter IS NOT NULL
    GROUP BY Year, ProductCategory, Quarter
),
-- This CTE pivots and calculates the quarterly totals for each combination of Year and ProductCategory.
-- It also includes a grand total column for the sum of all quarters.
QuarterlyTotals AS (
    SELECT 
        Year,
        ProductCategory,
        SUM(CASE WHEN Quarter = 'Q1' THEN TotalSales ELSE 0 END) AS Q1,
        SUM(CASE WHEN Quarter = 'Q2' THEN TotalSales ELSE 0 END) AS Q2,
        SUM(CASE WHEN Quarter = 'Q3' THEN TotalSales ELSE 0 END) AS Q3,
        SUM(CASE WHEN Quarter = 'Q4' THEN TotalSales ELSE 0 END) AS Q4,
        SUM(TotalSales) AS GrandTotal
    FROM SalesRevenuePivot
    GROUP BY Year, ProductCategory
),
-- This CTE calculates the grand total row for all products across all years.
GrandTotals AS (
    SELECT
        'Grand Total' AS Year,
        'Both Products' AS ProductCategory,
        SUM(Q1) AS Q1,
        SUM(Q2) AS Q2,
        SUM(Q3) AS Q3,
        SUM(Q4) AS Q4,
        SUM(GrandTotal) AS GrandTotal
    FROM QuarterlyTotals
)
SELECT * FROM QuarterlyTotals
UNION ALL
SELECT * FROM GrandTotals
ORDER BY Year, ProductCategory;
"""

file_path = 'solution.sql'

# Write the SQL script to the file
with open(file_path, 'w') as file:
    file.write(script)

dfd


In [5]:
"{}_{}".format(datetime.now().date(), datetime.now().time()) 

'2024-05-12_11:53:27.815121'

In [4]:
datetime.now().time()

datetime.time(11, 52, 21, 302993)

In [18]:
pd.read_parquet('output/parquet_1_time_2024-05-12 11:33:38.381868.gzip')

,studies,nextPageToken,pipeline_start_timestamp,pipeline_run_id
0,{'derivedSection': {'conditionBrowseModule': {...,NF0g5JuEk_Q,2024-05-12 11:33:38.381868,960463de-0060-4a3d-a736-888a730e0703
1,{'derivedSection': {'conditionBrowseModule': {...,NF0g5JuEk_Q,2024-05-12 11:33:38.381868,960463de-0060-4a3d-a736-888a730e0703
2,{'derivedSection': {'conditionBrowseModule': {...,NF0g5JuEk_Q,2024-05-12 11:33:38.381868,960463de-0060-4a3d-a736-888a730e0703
3,{'derivedSection': {'conditionBrowseModule': {...,NF0g5JuEk_Q,2024-05-12 11:33:38.381868,960463de-0060-4a3d-a736-888a730e0703
4,{'derivedSection': {'conditionBrowseModule': {...,NF0g5JuEk_Q,2024-05-12 11:33:38.381868,960463de-0060-4a3d-a736-888a730e0703
5,{'derivedSection': {'conditionBrowseModule': {...,NF0g5JuEk_Q,2024-05-12 11:33:38.381868,960463de-0060-4a3d-a736-888a730e0703
6,{'derivedSection': {'conditionBrowseModule': {...,NF0g5JuEk_Q,2024-05-12 11:33:38.381868,960463de-0060-4a3d-a736-888a730e0703
7,{'derivedSection': {'conditionBrowseModule': {...,NF0g5JuEk_Q,2024-05-12 11:33:38.381868,960463de-0060-4a3d-a736-888a730e0703
8,{'derivedSection': {'conditionBrowseModule': {...,NF0g5JuEk_Q,2024-05-12 11:33:38.381868,960463de-0060-4a3d-a736-888a730e0703
9,{'derivedSection': {'conditionBrowseModule': {...,NF0g5JuEk_Q,2024-05-12 11:33:38.381868,960463de-0060-4a3d-a736-888a730e0703
